# 2장 시계열 데이터의 발견 및 다루기

In [1]:
import pandas as pd

YearJoined = pd.read_csv('Ch02/data/year_joined.csv')

YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [2]:
emails = pd.read_csv('Ch02/data/emails.csv')

emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [3]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [4]:
# (max(emails[emails.user==998].week) - min(emails[emails.user==998].week)).days/7

In [5]:
max(emails[emails.user==998].week)

'2018-05-28 00:00:00'

In [6]:
import datetime
datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')

datetime.datetime(2018, 5, 28, 0, 0)

In [7]:
(datetime.datetime.strptime(max(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(min(emails[emails.user==998].week), '%Y-%m-%d %H:%M:%S')).days/7

25.0

In [8]:
emails[emails.user==998].shape

(24, 3)

In [9]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [10]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()

all_email.columns=['week', 'user', 'emailsOpened']

In [11]:
all_email[all_email.user==998].sort_values('week')

,week,user,emailsOpened
16708,2015-02-09 00:00:00,998.0,0.0
64140,2015-02-16 00:00:00,998.0,0.0
15091,2015-02-23 00:00:00,998.0,0.0
44197,2015-03-02 00:00:00,998.0,0.0
28027,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
81927,2018-04-30 00:00:00,998.0,3.0
79771,2018-05-07 00:00:00,998.0,3.0
30183,2018-05-14 00:00:00,998.0,3.0
69530,2018-05-21 00:00:00,998.0,3.0


In [12]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

cutoff_dates = cutoff_dates.reset_index()

In [13]:
for _, row in cutoff_dates.iterrows():
  member = row['user']
  start_date = row['min']
  end_date = row['max']

  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)

  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)

<ipython-input-13-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)
<ipython-input-13-8325b9330e44>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)
<ipython-input-13-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week < start_date].index, inplace=True)
<ipython-input-13-8325b9330e44>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == member][all_email.week > end_date].index, inplace=True)
<ipython-input-13-8325b9330e44>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.

In [14]:
donations = pd.read_csv('Ch02/data/donations.csv')

In [15]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())

<ipython-input-15-a6d693572295>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())


In [16]:
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())

<ipython-input-16-f53e5220f04a>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(lambda df: df.amount.resample('W-MON').sum().dropna())


In [17]:
agg_don = donations.groupby('user').apply(
    lambda df: df.amount.resample('W-MON').sum().loc[lambda x: x != 0]
)

<ipython-input-17-e0f3e960d83f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  agg_don = donations.groupby('user').apply(


In [18]:
agg_donations = pd.DataFrame(agg_don).reset_index()

In [19]:
agg_donations

,user,timestamp,amount
0,0.0,2015-03-30,25.0
1,0.0,2015-08-31,50.0
2,0.0,2016-07-11,50.0
3,0.0,2017-11-13,25.0
4,1.0,2016-05-09,50.0
...,...,...,...
2607,992.0,2018-05-21,50.0
2608,993.0,2017-11-06,50.0
2609,995.0,2016-09-19,1000.0
2610,995.0,2017-10-02,1000.0


In [20]:
for i, j in all_email.groupby('user'):
  print(i, j)

1.0                       week  user  emailsOpened
0      2017-10-02 00:00:00   1.0           3.0
539    2016-11-14 00:00:00   1.0           3.0
1078   2015-09-21 00:00:00   1.0           2.0
1617   2017-02-06 00:00:00   1.0           3.0
2156   2017-10-23 00:00:00   1.0           3.0
...                    ...   ...           ...
90552  2015-07-06 00:00:00   1.0           0.0
91091  2015-08-31 00:00:00   1.0           2.0
91630  2016-07-25 00:00:00   1.0           3.0
92169  2016-05-30 00:00:00   1.0           3.0
92708  2016-05-23 00:00:00   1.0           3.0

[153 rows x 3 columns]
3.0                       week  user  emailsOpened
52823  2018-04-09 00:00:00   3.0           1.0
56057  2018-03-26 00:00:00   3.0           2.0
67376  2018-03-19 00:00:00   3.0           3.0
68454  2018-04-23 00:00:00   3.0           2.0
73844  2018-04-16 00:00:00   3.0           2.0
84624  2018-04-02 00:00:00   3.0           1.0
86241  2018-03-05 00:00:00   3.0           3.0
88936  2018-03-12 00:00:00  

In [21]:
all_email

,week,user,emailsOpened
0,2017-10-02 00:00:00,1.0,3.0
2,2017-10-02 00:00:00,5.0,1.0
3,2017-10-02 00:00:00,6.0,0.0
4,2017-10-02 00:00:00,9.0,3.0
6,2017-10-02 00:00:00,14.0,0.0
...,...,...,...
93221,2016-05-23 00:00:00,940.0,1.0
93226,2016-05-23 00:00:00,951.0,3.0
93229,2016-05-23 00:00:00,956.0,0.0
93230,2016-05-23 00:00:00,958.0,1.0


In [22]:
all_email['week'] = pd.to_datetime(all_email['week'])
all_email['week'] = all_email['week'].dt.to_period('W-MON').dt.start_time

agg_donations['timestamp'] = agg_donations['timestamp'].dt.to_period('W-MON').dt.start_time

In [23]:
merged_df = pd.DataFrame()

for member, member_email in all_email.groupby('user'):
  member_donations = agg_donations[agg_donations.user==member]

  member_donations.set_index('timestamp', inplace=True)

  member_email = all_email[all_email.user==member]
  print(member_email)
  member_email.week = pd.to_datetime(member_email.week)
  member_email.set_index('week', inplace=True)

  df = pd.merge(member_email, member_donations, how='left', left_index=True, right_index=True)
  print(df)

  df = df.fillna(0)

  df['member'] = df.user_x
  merged_df = pd.concat([merged_df, (df.reset_index()[['member', 'week', 'emailsOpened', 'amount']])])

            week  user  emailsOpened
0     2017-09-26   1.0           3.0
539   2016-11-08   1.0           3.0
1078  2015-09-15   1.0           2.0
1617  2017-01-31   1.0           3.0
2156  2017-10-17   1.0           3.0
...          ...   ...           ...
90552 2015-06-30   1.0           0.0
91091 2015-08-25   1.0           2.0
91630 2016-07-19   1.0           3.0
92169 2016-05-24   1.0           3.0
92708 2016-05-17   1.0           3.0

[153 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2017-09-26     1.0           3.0     NaN     NaN
2016-11-08     1.0           3.0     NaN     NaN
2015-09-15     1.0           2.0     NaN     NaN
2017-01-31     1.0           3.0     NaN     NaN
2017-10-17     1.0           3.0     NaN     NaN
...            ...           ...     ...     ...
2015-06-30     1.0           0.0     NaN     NaN
2015-08-25     1.0           2.0     NaN     NaN
2016-07-19     1.0           3.0     NaN  

<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            user_x  emailsOpened  user_y  amount
week                                            
2018-02-27    66.0           1.0     NaN     NaN
            week  user  emailsOpened
32    2017-09-26  69.0           3.0
2188  2017-10-17  69.0           3.0
2727  2017-11-07  69.0           3.0
3266  2017-03-14  69.0           3.0
4344  2017-06-06  69.0           3.0
...          ...   ...           ...
84655 2018-03-27  69.0           3.0
85194 2017-05-30  69.0           3.0
85733 2017-05-23  69.0           3.0
86272 2018-02-27  69.0           3.0
88967 2018-03-06  69.0           3.0

[68 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2017-09-26    69.0           3.0     NaN     NaN
2017-10-17    69.0           3.0     NaN     NaN
2017-11-07    69.0           3.0    69.0   500.0
2017-03-14    69.0           3.0     NaN     NaN
2017-06-06    69.0           3.0     NaN     NaN
...            ...           ...     ...   

<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
66    2017-09-26  128.0           3.0
605   2016-11-08  128.0           3.0
1683  2017-01-31  128.0           3.0
2222  2017-10-17  128.0           3.0
2761  2017-11-07  128.0           3.0
...          ...    ...           ...
85767 2017-05-23  128.0           3.0
86306 2018-02-27  128.0           3.0
87384 2016-08-30  128.0           3.0
87923 2016-10-11  128.0           3.0
89001 2018-03-06  128.0           3.0

[93 rows x 3 columns]
            user_x  emailsOpened  user_y  amount
week                                            
2017-09-26   128.0           3.0     NaN     NaN
2016-11-08   128.0           3.0     NaN     NaN
2017-01-31   128.0           3.0     NaN     NaN
2017-10-17   128.0           3.0     NaN     NaN
2017-11-07   128.0           3.0     NaN     NaN
...            ...           ...     ...     ...
2017-05-23   128.0           3.0     NaN     NaN
2018-02-27   128.0           3.0     NaN     NaN
2016-08-30   128.0           3.

...

<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
2120  2017-01-31  923.0           2.0
3737  2017-03-14  923.0           0.0
5354  2017-02-07  923.0           2.0
23141 2017-01-24  923.0           0.0
31765 2017-02-28  923.0           0.0
35538 2017-01-10  923.0           1.0
37694 2017-03-07  923.0           0.0
40928 2017-02-14  923.0           1.0
43084 2017-03-21  923.0           1.0
58176 2017-01-17  923.0           1.0
58715 2017-01-03  923.0           2.0
71651 2016-12-27  923.0           1.0
81353 2017-02-21  923.0           0.0
            user_x  emailsOpened  user_y  amount
week                                            
2017-01-31   923.0           2.0     NaN     NaN
2017-03-14   923.0           0.0     NaN     NaN
2017-02-07   923.0           2.0     NaN     NaN
2017-01-24   923.0           0.0     NaN     NaN
2017-02-28   923.0           0.0     NaN     NaN
2017-01-10   923.0           1.0     NaN     NaN
2017-03-07   923.0           0.0     NaN     NaN
2017-02-14   923.0         

<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)
<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

            week   user  emailsOpened
7545  2018-02-06  998.0           3.0
12935 2018-01-16  998.0           3.0
16169 2018-02-20  998.0           2.0
19403 2018-01-02  998.0           3.0
26410 2017-11-28  998.0           1.0
28566 2017-12-26  998.0           3.0
30183 2018-05-08  998.0           3.0
36112 2018-05-22  998.0           3.0
51743 2018-01-23  998.0           2.0
53360 2018-04-03  998.0           3.0
54977 2018-01-30  998.0           3.0
56594 2018-03-20  998.0           2.0
63062 2017-12-19  998.0           0.0
64679 2018-01-09  998.0           2.0
67913 2018-03-13  998.0           2.0
68991 2018-04-17  998.0           0.0
69530 2018-05-15  998.0           3.0
74381 2018-04-10  998.0           3.0
77076 2017-12-05  998.0           3.0
79771 2018-05-01  998.0           3.0
80310 2017-12-12  998.0           3.0
80849 2018-02-13  998.0           3.0
81927 2018-04-24  998.0           3.0
85161 2018-03-27  998.0           3.0
86778 2018-02-27  998.0           2.0
89473 2018-0

<ipython-input-23-ce860758a3f2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_email.week = pd.to_datetime(member_email.week)


In [24]:
merged_df.loc[lambda x: x.amount != 0]

,member,week,emailsOpened,amount
50,1.0,2016-05-10,3.0,50.0
63,1.0,2016-05-03,3.0,50.0
93,1.0,2017-01-17,3.0,75.0
21,5.0,2017-09-19,1.0,25.0
33,5.0,2017-07-25,2.0,50.0
...,...,...,...,...
62,987.0,2017-07-25,3.0,250.0
6,993.0,2017-10-31,3.0,50.0
0,995.0,2017-09-26,3.0,1000.0
45,995.0,2016-09-13,3.0,1000.0


In [25]:
merged_df = merged_df.sort_values(by='week', ascending=True)

In [26]:
df = merged_df[merged_df.member == 998]

df['target'] = df.amount.shift(1)
df = df.fillna(0)
df

<ipython-input-26-a2aec95fca91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df.amount.shift(1)


,member,week,emailsOpened,amount,target
4,998.0,2017-11-28,1.0,0.0,0.0
18,998.0,2017-12-05,3.0,0.0,0.0
20,998.0,2017-12-12,3.0,0.0,0.0
12,998.0,2017-12-19,0.0,0.0,0.0
5,998.0,2017-12-26,3.0,0.0,0.0
3,998.0,2018-01-02,3.0,50.0,0.0
13,998.0,2018-01-09,2.0,0.0,50.0
1,998.0,2018-01-16,3.0,0.0,0.0
8,998.0,2018-01-23,2.0,0.0,0.0
10,998.0,2018-01-30,3.0,0.0,0.0
